# **Modelling and Evaluation**

## Objectives

**Perform Business requirement 2 user story tasks: model selection, pipeline creation, hyperparameter tuning, model evaluation.**
* Create initial data cleaning and engineering pipeline using information from previous notebooks.
* Create initial modelling and evaluation pipeline using information from previous notebooks.
* Find best model candidate.
* Optimise chosen model through tuning and feature selection using feature importance. 
* Evaluate the model performance using performance metrics.
* Successfully achieve $R^2 \ge 0.75$ for the final model, to satisfy the client's success criteria, and thereby satisfy business requirement 2.


## Inputs
* house prices dataset: outputs/datasets/collection/house_prices.csv.
* Information regarding the steps to include in the various pipelines, as indicated in the conclusion sections of the data cleaning and feature engineering notebooks.
* Outlier indices list: src/ml/outlier_indices.pkl

## Outputs
* train set: src/ml/train_set_df.csv
* test set: src/ml/test_set_df.csv
* pickled data cleaning and engineering pipeline: src/ml/data_cleaning_and_feature_engineering_pipeline.pkl
* pickled model pipeline: src/ml/model_pipeline.pkl
* feature importances CSV: src/ml/feature_importances_df.csv
* model performances CSV: src/ml/model_pipeline.pkl

---

## Change working directory

Working directory changed to its parent folder.

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
os.getcwd()

---

## Load house price dataset

In [ ]:
import pandas as pd

house_prices_df = pd.read_csv(filepath_or_buffer='outputs/datasets/collection/house_prices.csv')

---

## Removing known outliers from the whole dataset

Loading outlier indices list

In [ ]:
import joblib
outlier_indices = joblib.load('src/ml/outlier_indices.pkl')
outlier_indices

Removing the instances

In [ ]:
house_prices_df.drop(labels=outlier_indices, inplace=True)

---

## Create data cleaning and feature engineering pipeline

In [ ]:
import numpy as np
import src.ml.transformers_and_functions as tf
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from feature_engine.selection import SmartCorrelatedSelection, DropFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from feature_engine.transformation import PowerTransformer

In [ ]:

def data_cleaning_and_feature_engineering():
    """
    Constructs and returns data cleaning and feature engineering pipeline.
    """
    # variables for defining pipeline
    estimator = DecisionTreeRegressor(min_samples_split=10, min_samples_leaf=5, random_state=30)
    # Orginally intended to include the categories parameter used previously for OrdinalEncoder in the
    # feature engineering notebook. However causes problems when not all category options are
    # present in the train or test set. Will use the 'auto' option instead.
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype='int64')
    encoder.set_output(transform='pandas')
    min_max_scaler = MinMaxScaler()
    min_max_scaler.set_output(transform='pandas')

    pipeline = Pipeline([
                        # Data cleaning:
                        # Missing value imputation:
                        ('IndependentKNNImputer', tf.IndependentKNNImputer()),
                        ('EqualFrequencyImputer', tf.EqualFrequencyImputer()),
                        #feature engineering:
                        # encoding:
                        ('OrdinalEncoder', ColumnTransformer(transformers=[('encoder', encoder, ['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'])],
                                                      remainder='passthrough', n_jobs=-1, verbose_feature_names_out=False)),
                        # feature scaling:
                        ('Scaling', ColumnTransformer(transformers=[('Power', PowerTransformer(), ['YearBuilt', 'GrLivArea', 'BsmtFinSF1'])],
                                                    remainder=min_max_scaler, n_jobs=-1, verbose_feature_names_out=False)),
                        # feature number reduction
                        ('CompositeSelectKBest', tf.CompositeSelectKBest()),
                        ('SmartCorrelatedSelection', SmartCorrelatedSelection(method='spearman',
                                                                              threshold=0.8, selection_method='model_performance',
                                                                              estimator=estimator, scoring='r2', cv=5))
                        ])
                        
    pipeline.set_output(transform='pandas')
    return pipeline

As commented inside the data cleaning and feature engineering function above, it was intended to manually specify the ordinal encoding mapping using ordered arrays, as was done in the feature engineering notebook. However the train and test sets might not have all feature options for all features. For example for the train set the feature 'KitchenQual' does not have the value 'Po'. Therefore the encoding will be done automatically, and consequently the natural ranking of the feature values for a feature may not be preserved. 

In [ ]:
data_cleaning_and_feature_engineering_pipeline = data_cleaning_and_feature_engineering()
data_cleaning_and_feature_engineering_pipeline.set_output(transform='pandas')

## Split dataset

When random splitting the dataset into train and test subsets, as determined by the value of random_state, the quality of split and therefore the model performance on the test set
will vary, the degree of which may be impacted by the inherent variability of the final model. If the train and or test sets are not representative of the original parent dataset, then the model performance may be adversely affected. The assumption being that the original dataset, a sample itself, is representative of the fixed parent population if one exists.
The sizes of the train and test sets also indirectly impact the likelihood that a random split is representative, namely the more data in each the more likely it is; thus the larger the parent set as a whole, the less likely the random split will affect model performance significantly.

Stratification by using a dataset variable may help to ensure the train and test sets are more representative, however if there are many uncorrelated variables in the dataset, skewed splits may still occur.

To assess the quality of a random split, plots and test statistics (with or without significance values) can be used. As there are countably infinite possible random_state values, even though the number of distinguishable splits may be finite, it may not be practical to discover the best split; however at the very least the train and test sets should be representative of the original dataset.

The other factor that may be impact an aspect of split quality is the relative sizes of the train and test sets. The larger the train set, the more likely the model will learn any patterns in the data, and thus be able to make better predictions. The larger the test set size the greater the confidence in the accuracy and reliability of the model performance metric scores. Thus naturally there is a trade-off between having an optimally large test set or train set, at least for small datasets. For sufficiently large datasets, it is possible to have both an optimally large train and test set.

The house prices dataset size is fairly small (~1500), and the random split is likely to affect model performance. Stratification by the sale price target will hopefully ensure a similar distribution for it in the train and test sets, and thus also for other strongly correlated features. Will perform 100 splits and assess the quality of each, and pick the best. 

**Creating the sale price bins needed for stratification**

In [ ]:
import scipy
from sklearn.model_selection import train_test_split

In [ ]:
sale_price_bin_series = pd.cut(x=house_prices_df['SalePrice'], bins=10)
print(sale_price_bin_series)
house_prices_df['sale price bin'] = sale_price_bin_series
house_prices_df[['SalePrice', 'sale price bin']].head(10)

### Performing the 100 different random splits

Identifying the continuous and discrete dataset variables, and partitioning the dataset accordingly

In [ ]:
continuous_variables = ['1stFlrSF',
                        '2ndFlrSF',
                        'BsmtFinSF1',
                        'BsmtUnfSF',
                        'EnclosedPorchSF',
                        'GarageArea',
                        'GrLivArea',
                        'LotArea',
                        'LotFrontage',
                        'MasVnrArea',
                        'OpenPorchSF',
                        'TotalBsmtSF',
                        'WoodDeckSF',
                        'SalePrice']
continuous_variables_df = house_prices_df[continuous_variables]

In [ ]:
discrete_variables_df = house_prices_df.drop(continuous_variables, axis=1)
discrete_variables = discrete_variables_df.columns.values.tolist()
discrete_variables.remove('sale price bin')
discrete_variables

The quality of each split will be assessed by testing how representative the train and test sets are of the whole dataset. For each test and train distribution resulting from a split, chi-squared significance tests will be used for discrete variables, and the two-sample Kolmogorov-Smirnov significance test for continuous variables. A significance level of 50% will be used to set a low bar for rejecting the null hypothesis that the distribution of a variable in the train/test set is the same as its distribution in the whole dataset.
For each split, for each subset, the number of variables that have distributions that make the null hypothesis true, will be counted. In addition the product of the individual p-values for each variable for both train and test sets combined will be generated. Collectively these metrics will allow the split quality to be estimated and ranked. It is likely the ranking of splits is not 100% correct due to the way which the p-value product is calculated (for example it assumes independent probabilities). In addition the model performance is not necessarily higher or more reliable for higher ranked splits. However the number of variables for which the null hypothesis is true should indicate whether a given split produces subsets that are fully representative of the whole dataset.

**Re-binning discrete variables that have bin frequencies less than 10 as per the requirements of the chi square test**

In [ ]:
variables_to_be_rebinned = []
for variable in discrete_variables:
    if not (discrete_variables_df[variable].value_counts(dropna=False) > 10).all():
        variables_to_be_rebinned.append(variable)
variables_to_be_rebinned

In [ ]:
binned_discrete_vars_df = discrete_variables_df.copy(deep=True)

Creating new bins for the required variables

In [ ]:
print(discrete_variables_df['BedroomAbvGr'].value_counts(dropna=False))
print(pd.cut(discrete_variables_df['BedroomAbvGr'], bins=[0,1,2,3,4,8]).value_counts(dropna=False))
binned_discrete_vars_df['BedroomAbvGr'] = pd.cut(discrete_variables_df['BedroomAbvGr'], bins=[0,1,2,3,4,8])

In [ ]:
print(discrete_variables_df['GarageYrBlt'].value_counts(dropna=False))
print(pd.qcut(discrete_variables_df['GarageYrBlt'], q=10).value_counts(dropna=False))
binned_discrete_vars_df['GarageYrBlt'] = pd.qcut(discrete_variables_df['GarageYrBlt'], q=10)

In [ ]:
print(discrete_variables_df['YearBuilt'].value_counts(dropna=False))
print(pd.qcut(discrete_variables_df['YearBuilt'], q=10).value_counts(dropna=False))
binned_discrete_vars_df['YearBuilt'] = pd.qcut(discrete_variables_df['YearBuilt'], q=10)

In [ ]:
print(discrete_variables_df['YearRemodAdd'].value_counts(dropna=False))
print(pd.cut(discrete_variables_df['YearRemodAdd'], bins=10).value_counts(dropna=False))
binned_discrete_vars_df['YearRemodAdd'] = pd.cut(discrete_variables_df['YearRemodAdd'], bins=10)

In [ ]:
print(discrete_variables_df['OverallCond'].value_counts(dropna=False))
print(pd.cut(discrete_variables_df['OverallCond'], bins=[0,3,4,5,6,7,8,9]).value_counts(dropna=False))
binned_discrete_vars_df['OverallCond'] = pd.cut(discrete_variables_df['OverallCond'], bins=[0,3,4,5,6,7,8,9])

In [ ]:
print(discrete_variables_df['OverallQual'].value_counts(dropna=False))
print(pd.cut(discrete_variables_df['OverallQual'], bins=[0,3,4,5,6,7,8,9,10]).value_counts(dropna=False))
binned_discrete_vars_df['OverallQual'] = pd.cut(discrete_variables_df['OverallQual'], bins=[0,3,4,5,6,7,8,9,10])

In [ ]:
binned_discrete_vars_df.head()

**Creating a dataframe to facilitate the random split quality comparison**

In [ ]:
index_array = [['number of distributions where the the null hypothesis is true', 'p value product'], ['parent-train', 'parent-test', 'total']]
multi_index = pd.MultiIndex(index_array, [np.append(np.repeat([0], 3), 1), np.append(np.tile([0,1,2], 1), 2)])
multi_index

In [ ]:
distribution_comparison_df = pd.DataFrame(columns=multi_index, dtype='float')
distribution_comparison_df

In [ ]:
seed = 0
while seed < 100:
    house_prices_copy_df = house_prices_df.copy(deep=True)
    house_prices_copy_df[(house_prices_df.drop(continuous_variables, axis=1)).columns] = binned_discrete_vars_df
    # performing the split for the current seed
    (train_set_df, test_set_df) = train_test_split(house_prices_copy_df, test_size=0.25, random_state=seed, stratify=house_prices_copy_df['sale price bin'])
    # dropping the sale price bin column from dataframes
    house_prices_copy_df.drop('sale price bin', axis=1, inplace=True)
    train_set_df.drop('sale price bin', axis=1, inplace=True)
    test_set_df.drop('sale price bin', axis=1, inplace=True)
    # partitioning the train and test sets by their continuous and discrete variables
    train_continuous_var_df = train_set_df[continuous_variables]
    train_discrete_var_df = train_set_df[discrete_variables]
    test_continuous_var_df = test_set_df[continuous_variables]
    test_discrete_var_df = test_set_df[discrete_variables]
    # creating an initial zero row for this seed in the distribution comparison dataframe
    distribution_comparison_df = pd.concat(objs=[distribution_comparison_df, pd.DataFrame(index=[seed], columns=multi_index, data=np.array([np.append(np.repeat(0, 3), 1)]))])

    # calculating the chi-squared test statistics for the discrete variables for each dataset comparison
    for column in discrete_variables:

        # parent-test distribution comparisons:
        
        # value proportion of the current variable in the whole dataset
        value_proportion_series = binned_discrete_vars_df[column].value_counts(dropna=False)/binned_discrete_vars_df.index.size
        # expected values for the current variable
        expected = (value_proportion_series*test_discrete_var_df.index.size)
        # observed values for the current variable
        observed = test_discrete_var_df[column].value_counts(dropna=False)
        # calculating the chi-squared statistic p-value for the current variable
        p_value = scipy.stats.chisquare(observed, expected)[1]
        # Assess significance value and update distribution_comparison df
        if p_value > 0.5:
            distribution_comparison_df.iloc[seed, 1] += 1
        distribution_comparison_df.iloc[seed, 3] *= p_value
       
        # parent-train distribution comparisons:

        observed = train_discrete_var_df[column].value_counts(dropna=False)
        expected = (value_proportion_series*train_discrete_var_df.index.size).values.tolist()
        observed = train_discrete_var_df[column].value_counts(dropna=False).values.tolist()
        p_value = scipy.stats.chisquare(observed, expected)[1]
        if p_value > 0.5:
            distribution_comparison_df.iloc[seed, 0] += 1
        distribution_comparison_df.iloc[seed, 3] *= p_value
        
    # calculating the KS test statistics for the continuous variables for each dataset comparison
    for column in continuous_variables:
        # parent-train distribution comparisons:

        # calculating the KS test statistic for the current variable
        p_value = scipy.stats.ks_2samp(continuous_variables_df[column], train_continuous_var_df[column])[1]
        # Assess significance value and update distribution_comparison df
        if p_value > 0.5:
            distribution_comparison_df.iloc[seed, 0] += 1
        distribution_comparison_df.iloc[seed, 3] *= p_value
        
        
        # parent-test distribution comparisons:

        p_value = scipy.stats.ks_2samp(continuous_variables_df[column], test_continuous_var_df[column])[1]
        if p_value > 0.5:
            distribution_comparison_df.iloc[seed, 1] += 1
        distribution_comparison_df.iloc[seed, 3] *= p_value

            
    seed += 1
# Calculating the total number of distributions for which the null hypothesis is true
distribution_comparison_df.iloc[:, 2] = distribution_comparison_df.iloc[:, 0] + distribution_comparison_df.iloc[:, 1]

In [ ]:
distribution_comparison_df.sort_values(by=[('number of distributions where the the null hypothesis is true', 'total'), ('p value product', 'total')],
                                       ascending=[False, False]).head(50)

**Thus using the above distribution comparison dataframe as a guide, a random_state value of 68 will be used to produce a representative split**.

In [ ]:
(train_set_df, test_set_df) = train_test_split(house_prices_df, test_size=0.25, random_state=68, stratify=house_prices_df['sale price bin'])

In [ ]:
house_prices_df.drop('sale price bin', axis=1, inplace=True)
train_set_df.drop('sale price bin', axis=1, inplace=True)
test_set_df.drop('sale price bin', axis=1, inplace=True)

Splitting the train and test sets in to their features and target

In [ ]:
x_train = train_set_df.drop('SalePrice', axis=1)
y_train = train_set_df['SalePrice']
x_test = test_set_df.drop('SalePrice', axis=1)
y_test = test_set_df['SalePrice']

---

## Create Scale target function

In [ ]:
def scale_target(y_fit, y_transform):
    """
    Scales target for a subset, having been trained on another subset.

    Args:
        y_fit: target values used for fitting.
        y_transform: target values transformed.

    Returns a tuple of the scaled target series, as well as the inverse transform.
    """
    y_fit = pd.DataFrame(data=y_fit)
    y_transform = pd.DataFrame(data=y_transform)
    min_max_scaler = MinMaxScaler()
    min_max_scaler.set_output(transform='pandas')
    min_max_scaler.fit(y_fit)
    y_transform = min_max_scaler.transform(y_transform)
    inverse_transform = min_max_scaler.inverse_transform

    return (y_transform.iloc[:, 0], inverse_transform)


### Saving the y_train set

In [ ]:
import joblib
import os

file_path = 'src/ml'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

In [ ]:
try:
    y_train.to_csv(f"{file_path}/y_train.csv")
except Exception as e:
    print(e)

## Creating functions needed to evaluate model performance

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def model_evaluation(x_train, y_train, x_test, y_test, pipelines):
    """
    Calculates predicted values for train and test sets. Prints statistics and plots assessing prediction accuracy.

    Args:
        x_train: train set feature data.
        x_test: test set feature data.
        y_train: actual train target values.
        y_test: actual test target values.
        pipelines: dictionary containing the data cleaning and engineering pipeline and the model pipeline.
    
    Returns a tuple of predictions, train statistics, and test statistics.
    """
    # transform train and test set target
    y_test = scale_target(y_train, y_test)[0]
    y_train, inverse_transform = scale_target(y_train, y_train)

    # transform test and train set features
    x_train = pipelines['data_cleaning_and_feature_engineering'].fit_transform(x_train, y_train)
    x_test = pipelines['data_cleaning_and_feature_engineering'].transform(x_test)
    
    # get target predictions
    predictions_train = pipelines['model'].fit(x_train, y_train).predict(x_train)
    predictions_test = pipelines['model'].predict(x_test)
    predictions = (predictions_train, predictions_test)

    # unscaling predictions
    y_train = pd.DataFrame(inverse_transform(pd.DataFrame(data=y_train)))
    y_test = pd.DataFrame(data=inverse_transform(pd.DataFrame(data=y_test)))
    predictions_train = pd.DataFrame(inverse_transform(pd.DataFrame(data=predictions[0])))
    predictions_test = pd.DataFrame(inverse_transform(pd.DataFrame(data=predictions[1])))

    predictions = (predictions_train, predictions_test)


    # gather summary performance statistics
    train_stats = model_evaluation_statistics(y_train, predictions_train)
    test_stats = model_evaluation_statistics(y_test, predictions_test)

    # print prediction vs actual plots
    model_evaluation_plots(y_train, y_test, predictions)
    
    return predictions, train_stats, test_stats

def model_evaluation_statistics(y, prediction):
    """
    Prints statistics assessing prediction accuracy.

    Args:
        y: actual values array-like
        prediction: predicted values array-like.
    
    Returns a list of statistics.
    """
    statistics = [r2_score(y, prediction).round(3), mean_absolute_error(y, prediction).round(3),
                  mean_squared_error(y, prediction).round(3), np.sqrt(mean_squared_error(y, prediction)).round(3)]
    print('R2 Score:', statistics[0])
    print('Mean Absolute Error:', statistics[1])
    print('Mean Squared Error:', statistics[2])
    print('Root Mean Squared Error:', statistics[3])
    print("\n")

    return statistics


def model_evaluation_plots(y_train, y_test, predictions, alpha_scatter=0.5):
    """
    Plots scatterplots including a line of perfect fit, for test and train actual values vs predicted values.

    Args:
        y_train: actual train target values.
        y_test: actual test target values.
        predictions: tuple of (predicted-train-target, predicted-test-target).
    """
    # plotting scatterplots with a perfect fit line
    prediction_train = predictions[0]
    prediction_test = predictions[1]
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 8), tight_layout=True)
    sns.scatterplot(x=y_train.iloc[:, 0], y=prediction_train.iloc[:, 0], alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train.iloc[:, 0], y=y_train.iloc[:, 0], color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    sns.scatterplot(x=y_test.iloc[:, 0], y=prediction_test.iloc[:, 0], alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test.iloc[:, 0], y=y_test.iloc[:, 0], color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    plt.show()

---

## Model Grid Search CV

Initially a search will be done to find the most suitable algorithm using sklearn's 'GridSearchCV', using only the default hyperparameters for each algorithm.
Hyperparmeter tuning will then be performed for this best candidate algorithm, again using 'GridSearchCV', but with multiple hyperparameter value combinations.

### Best algorithm search

Creating a search class to handle the searches.

In [ ]:
from sklearn.model_selection import GridSearchCV

# taken from code-Institute-Solutions/churnometer (https://github.com/Code-Institute-Solutions/churnometer)
class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = self.models[key]

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

Preparing parameters for conducting search.

Creating a dictionary of candidate models.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor

models = {'LinearRegression': LinearRegression(),
          'DecisionTreeRegressor': DecisionTreeRegressor(random_state=30),
          'RandomForestRegressor': RandomForestRegressor(random_state=30),
          'ExtraTreeRegressor': ExtraTreeRegressor(random_state=30),
          'AdaBoostRegressor': AdaBoostRegressor(random_state=30),
          'BaggingRegressor': BaggingRegressor(random_state=30)}

Defining the model parameters for each model; in this case there are no specified parameters meaning the default parameters will be used only as intended.

In [ ]:
default_model_params = {'LinearRegression': {},
                        'DecisionTreeRegressor': {},
                        'RandomForestRegressor': {},
                        'ExtraTreeRegressor': {},
                        'AdaBoostRegressor': {},
                        'BaggingRegressor': {}}

Applying the data cleaning and engineering pipeline to a copy of the train set features, and scaling a copy of the train target

In [ ]:
x_train_copy = x_train.copy(deep=True)
y_train_copy = y_train.copy(deep=True)

In [ ]:
y_train_copy = scale_target(y_train_copy, y_train_copy)[0]

In [ ]:
x_train_copy = data_cleaning_and_feature_engineering_pipeline.fit(x_train_copy, y_train_copy).transform(x_train_copy)

Performing the search with the created parameters.

In [ ]:
search = HyperparameterOptimizationSearch(models, default_model_params)
search.fit(x_train_copy, y_train_copy, scoring='r2', cv=5, n_jobs=-1)

In [ ]:
grid_search_results_summary, grid_search_pipelines = search.score_summary()

In [ ]:
grid_search_results_summary

All estimators have small or fairly small (<0.1*mean) standard deviations. The best max and mean score was achieved by the 'RandomForestRegressor', closely followed by the 'BaggingRegressor'. The top four estimators all achieved mean scores better than the desired minimum of $R^2=0.75$.

All things considered the 'RandomForestRegressor' seems to be the best model candidate. Hyperparameter tuning will now be performed for this model using GridSearchCV with multiple
hyperparameter combinations, aided by the use of the HyperparameterOptimizationSearch class.


### Creating MLModel class to handle model tuning and performance evaluation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
class MLModel():
    """
    Handles candidate model hyperparameter tuning searches, and model performance evaluation.
    
    For an initialised estimator, train and test dataset, and data cleaning and engineering pipeline,
    multiple different tuning searches can be performed using provided model parameters. For each search
    the best regressor is identified, and the feature importances for the best fitted model can be extracted.
    The best regressor model performance can also be evaluated. All performed searches and other related information
    are stored, and are acessible through label indices.

    Attributes:
        x_train: feature training dataframe.
        y_train: target training dataframe.
        x_test: feature test dataframe.
        y_test: target test dataframe.
        cleaning_engineering_pipeline: data cleaning and engineering pipeline instance.
        estimator_name: name of the estimator used in the model.
        estimator_instance: instance of the estimator used in the model.
        indices: list of all performed search indices.
        model_params: dictionary of model parameters used in each tuning search performed.
        searches: dictionary of search instances used in the tuning searches performed.
        best_regressors: dictionary of the best regressor instances identified from the searches performed.
        model_pipelines: dictionary of the created model pipelines, created from the identified best regressors.
        feature_importances: dictionary of feature importance dataframes, produced from each best fitted model.
        model_performances_df: dataframe containing as rows all the performance metrics of each of the best regressors evaluated on the train and test sets.
    """
    def __init__(self, cleaning_engineering_pipeline, estimator_name, estimator_instance, x_train, y_train, x_test, y_test):
        self.cleaning_engineering_pipeline = cleaning_engineering_pipeline
        self.cleaning_engineering_pipeline.set_output(transform='pandas')
        self.estimator_name = estimator_name
        self.estimator_instance = estimator_instance
        self.indices = []
        self.model_params = {}
        self.searches = {}
        self.best_regressors = {}
        self.model_pipelines = {}
        self.feature_importances = {}
        self.model_performances_df = pd.DataFrame(columns=pd.MultiIndex.from_product([['train', 'test'], ['R2', 'MAE', 'MSE', 'RMSE']]), dtype='float64')
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def transform_training_data(self):
        """
        Fits and transforms the training data using the cleaning_engineering_pipeline property value and the scale target function.

        Returns:
            The transformed data as a tuple (x_train, y_train).
        """
        y_train = scale_target(self.y_train, self.y_train)[0]
        x_train = self.x_train.copy(deep=True)
        x_train = self.cleaning_engineering_pipeline.fit(x_train, y_train).transform(x_train)
        return (x_train, y_train)
    
    def tune(self, model_params):
        """
        Performs a hyperparameter tuning 'GridSearchCV' search using the specified model_params.

        Performs the search using the training data, pipeline and estimator properties. Stores the search and model_params
        by updating the respective properties with a new uniquely indexed entry. Also extracts and stores the best regressor
        and its corresponding pipeline and feature importance, by updating the relevant instance properties.

        Args:
            model_params (dict): 'model_params' parameter for the 'HyperparameterOptimizationSearch' class.   
        """
        search = HyperparameterOptimizationSearch({self.estimator_name: self.estimator_instance}, model_params)
        x_train, y_train = self.transform_training_data()
        search.fit(x_train, y_train, scoring='r2', cv=5, n_jobs=-1)
        grid_search_results_summary, grid_search_pipelines = search.score_summary()
        # check uniqueness
        if model_params not in self.model_params.values():
            index = str(len(self.searches))
            # update indices
            self.indices.append(index)
            # update searches
            self.searches.update({index: search})
            # update model params
            self.model_params.update({index: model_params})
            # update best regressors
            self.best_regressors.update({index: grid_search_pipelines[self.estimator_name].best_estimator_})
            # update pipelines
            self.model_pipelines.update({index: Pipeline([(self.estimator_name, self.best_regressors[index])])})
            # update feature importance dict
            counter = -1
            while True:
                if hasattr(self.cleaning_engineering_pipeline[counter], 'get_feature_names_out'):
                    break
                counter -=1
            pipeline_features_out = self.cleaning_engineering_pipeline[counter].get_feature_names_out()
            regressor_feature_importances = self.best_regressors[index].feature_importances_
            feature_importances_df = pd.DataFrame(data=regressor_feature_importances, index=pipeline_features_out, columns=['importance']).sort_values(by='importance', ascending=False)
            self.feature_importances.update({index: feature_importances_df})
        else:
            print('Search performed previously.')
    
    def extract_feature_importance(self, index=None, print_plot=True):
        """
        Retrieves the feature importance for the best fitted model produced from tuning. It is printed as a barplot/dataframe and returned as a dataframe.

        Args:
            index (integer): list index of a performed hyperparameter tuning search. Default value is the index of the last performed search.
            print_plot (boolean): controls whether the barplot and dataframe are pirnted. Default value True.
        
        Returns:
            A feature importance dataframe for the best fitted model resulting from the stored tuning search matching the index.
        
        Raises:
            KeyError when the index is not found.
        """
        if index == None:
            index = len(self.searches) - 1
        try:
            feature_importances_df = self.feature_importances[str(index)]
            if print_plot:
                print(f'Feature importance for search index {index}:')
                print(feature_importances_df)
                # print plot
                fig, ax = plt.subplots(tight_layout=True, figsize=(13,5))
                ax.set_title('Feature Importance')
                sns.barplot(data=feature_importances_df, x=feature_importances_df.index, y=feature_importances_df['importance'], ax=ax)
                plt.show()
        except KeyError:
            print(f'There is no search matching the Index:{index}. Searches with indices in the range 0-{len(self.searches) - 1} currently exist.')
        else:
            return feature_importances_df

    def evaluate_model_performance(self, index=None ):
        """
        Evaluates the performance of the best fitted model produced from the indexed tuning search on the train and test data.
        
        Prints performance metrics and prediction scatterplots. Updates the model performances dataframe property by adding
        the model performance statistics as a new row.

        Args:
            index (integer): list index of a performed hyperparameter tuning search. Default value is the index of the last performed search.
        
        Raises:
            KeyError when the index is not found.
        """
        if index == None:
            index = len(self.searches) - 1
        
        try:
            predictions, train_stats, test_stats = model_evaluation(self.x_train, self.y_train, self.x_test, self.y_test,
                                                                    {'data_cleaning_and_feature_engineering': self.cleaning_engineering_pipeline,
                                                                    'model': self.model_pipelines[str(index)]})
            if index not in self.model_performances_df.index.to_list():
                new_entry_df = pd.DataFrame(dtype='float64', index=[index],
                                            data={('train', 'R2'): train_stats[0], ('train', 'MAE'): train_stats[1],
                                                    ('train', 'MSE'): train_stats[2], ('train', 'RMSE'): train_stats[3],
                                                    ('test', 'R2'): test_stats[0], ('test', 'MAE'): test_stats[1],
                                                    ('test', 'MSE'): test_stats[2], ('test', 'RMSE'): test_stats[3]})
                self.model_performances_df = pd.concat([self.model_performances_df, new_entry_df], ignore_index=False)
                self.model_performances_df.sort_values(by=[('test', 'R2')], inplace=True, ascending=False)
        except KeyError:
            print(f'There is no search matching the Index:{index}. Searches with indices in the range 0-{len(self.searches) - 1} currently exist.')

### Chosen best model candidate hyperparameter tuning

Choosing the model hyperparameter combinations

There are 7 hyperparamters that will be tuned:

* max_depth
* max_leaf_nodes
* min_samples_split
* min_samples_leaf
* n_estimators
* max_features
* max_samples

The ultimate goal is to avoid under-fitting and over-fitting the train set, leading to either high bias and low variance or low bias but high variance.
The other factor to consider is computation cost/time and complexity, with more complex models or higher values of hyperparameters, such as for max_depth, taking a lot more time to compute for the same computation power.

Ultimately there is a trade-off that must be decided subject to constraints and the success metric criteria.


Of the 7 parameters selected for tuning, some will counteract/limit the effects of each other for certain values. For example increasing max depth, everything else held constant, would lead to more nodes/layers in the tree, provided the min_samples_split/max_leaf_nodes/min_samples_split are not limiting more nodes/layers being formed.

What's more many of the parameters possess threshold values, either side of which the model performance on either or the train and test increases, decreases or plateaus.
Time permitting ideally a large range (small step-size) of parameters would be trialed and validation curves, for a single or group of parameters, plotted to discover the optimum values.

For this project, 3 values will be selected for each parameter and the impact of the various combinations assessed through using GridSearchCV via the HyperparameterOptimizationSearch class.

**max_depth**: the maximum number of layers

* As a starting point, the number of features will be used, corresponding to a possible tree structure where each feature is used once (assuming max_features matches) to
split a node. Will then take a value, half this, twice this, and two values in between. Hopefully this will indicate the rough location of a threshold value.
* So values [5,10,20].

**max_leaf_nodes**: the maximum number of terminal nodes. This will influence the number of nodes that can be split.

* If every node in every layer is split the number of nodes (n) increases with the depth at a rate $2^{n-1}$. If every node is split ending in leaves for a given depth n,
then the number of leaves will be $2^{n}$.
* The higher the number of leaf nodes, the more nodes that can be split, the greater the complexity/computation. For n=10 the described structure would have 512 leaves. Also more leaves may lead to over-fitting.
* Will cap the leaves at ~25% of this value
* So considering the depth values chosen, will take values [32,130,250].

**min_samples_split**: minimum number of samples at a node to split. This will counteract the number of leaves and the tree depth.
* 1460 samples. Taking the scenario where on average each split equally divides the samples, then it would take $n=log(1460)/log(2)$ levels to give pure leaves, if every node is split. So roughly 11 levels. Again all pure leaves likely leads to overfitting.
* So in this scenario lets say leaves have 10% max of the 1460 samples, giving a min_sample_split of 8. Will use this as a starting point.
* Will choose values [4,8,64].

**min_samples_leaf**: the minimum samples needed for a node to be a leaf.
* At worst would want a leaf to have no more than 5% of the samples, and probably not 1 sample either.
* Will use values [5,35,65].

**n_estimators**: number of trees in the forest. The more the better up to a point where the performance plateaus. However more trees equals greater computation time.
* The default value is 100, so will use this as a guide.
* Will use values [50,150,250].

**max_features**: The max number of features in a random subset to be used in a tree. Again this increases with the number of features up to a point but tails off and decreases.
* Apparently a good value for this can be obtained using the sqrt(no. of features).
* Also according to the sklearn documentation values close to 100% of the features give good empirical results.
* will use values ['sqrt',0.66,1.0].

**max_samples**: The sample size of the subset of samples.
 * Apparently a larger sample size increases the performance, but saturates quickly, and that only a small fraction of the sample is needed generally to achieve this saturation.
 * Thus will try the values [0.15,0.33,0.5].



**Setting the model parameters using the chosen values.**

In [ ]:

model_params = {'RandomForestRegressor': {
    'max_depth': [5,10,20],
    'max_samples': [0.15,0.33,0.5],
    'max_features': ['sqrt',0.66,1.0],
    'n_estimators': [50,150,250],
    'min_samples_leaf': [5,35,65],
    'min_samples_split': [4,8,64],
    'max_leaf_nodes': [32,130,250]
}}

Originally attempted to use 5 values for each parameter, but the computation time was far too long.

Create the MLModel instance for the chosen estimator

In [ ]:
random_forest_v1 = MLModel(cleaning_engineering_pipeline=data_cleaning_and_feature_engineering(), estimator_name='RandomForestRegressor',
                           estimator_instance=RandomForestRegressor(random_state=30), x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

**Performing the hyperparameter tuning search**

In [ ]:
random_forest_v1.tune(model_params=model_params)

Displaying the top 10 fitted estimators

In [ ]:
grid_search_results_summary, grid_search_pipelines = random_forest_v1.searches[random_forest_v1.indices[-1]].score_summary()

In [ ]:
grid_search_results_summary.head(10)

**Retrieving the best hyperparameter combination.**

In [ ]:
best_model = grid_search_results_summary.iloc[0, 0]
best_model_params = grid_search_pipelines[best_model].best_params_
print('Best model:', best_model)
print('Best hyperparameter combination:', best_model_params)

In [ ]:
random_forest_v1.best_regressors[random_forest_v1.indices[-1]]

**Extracting the feature importances**.

In [ ]:
feature_importances_df = random_forest_v1.extract_feature_importance()

Can see that half of the features account for most of the importance, and that the feature 'OverallQual' dominates.

---

## Evaluating model on train and test sets

**Evaluating the model on the train and test sets**:

In [ ]:
random_forest_v1.evaluate_model_performance()

Can see from the scatter plots and the result statistics for both the train and test set, that the model predicts well the train and test set target values similarly: the $R^2$ value for the test set is close to that of the train set. The current model, for both the test and train sets achieves $R^2>0.75$, and so the success criterion is met.

---

## **Refitting the model with less features**

Clearly the model is biased towards the train set, and has high variance for this test set. Will try to refit the model with fewer features with the hope of reducing the likelihood of over-fitting to the train set.

Selecting the four features with the highest feature importance

In [ ]:
best_four_features = feature_importances_df.iloc[0:5,:].index.values.tolist()
features_to_drop = x_train.drop(best_four_features, axis=1).columns.tolist()
print('Best four features:', best_four_features)
print('features to drop:', features_to_drop)

edit data cleaning and feature engineering pipeline

In [ ]:

def data_cleaning_and_feature_engineering_v2():
    """
    Constructs and returns data cleaning and feature engineering pipeline.
    """
    # variables for defining pipeline
    estimator = DecisionTreeRegressor(min_samples_split=10, min_samples_leaf=5, random_state=30)
    # Orginally intended to include the categories parameter used previously for OrdinalEncoder in the
    # feature engineering notebook. However causes problems when not all category options are
    # present in the train or test set. Will use the 'auto' option instead.
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype='int64')
    encoder.set_output(transform='pandas')
    min_max_scaler = MinMaxScaler()
    min_max_scaler.set_output(transform='pandas')

    pipeline = Pipeline([
                        # Data cleaning:
                        # Missing value imputation:
                        ('IndependentKNNImputer', tf.IndependentKNNImputer()),
                        ('EqualFrequencyImputer', tf.EqualFrequencyImputer()),
                        #feature engineering:
                        # encoding:
                        ('OrdinalEncoder', ColumnTransformer(transformers=[('encoder', encoder, ['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'])],
                                                      remainder='passthrough', n_jobs=-1, verbose_feature_names_out=False)),
                        # feature scaling:
                        ('Scaling', ColumnTransformer(transformers=[('Power', PowerTransformer(), ['YearBuilt', 'GrLivArea', 'BsmtFinSF1'])],
                                                    remainder=min_max_scaler, n_jobs=-1, verbose_feature_names_out=False)),
                        # feature number reduction
                        ('DropFeatures', DropFeatures(features_to_drop=features_to_drop))
                        ])
                        
    pipeline.set_output(transform='pandas')
    return pipeline

Creating MLModel instance with the new data cleaning and engineering pipeline

In [ ]:
random_forest_v2 = MLModel(cleaning_engineering_pipeline=data_cleaning_and_feature_engineering_v2(), estimator_name='RandomForestRegressor',
                           estimator_instance=RandomForestRegressor(random_state=30), x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

Redo tuning with best four features only

In [ ]:
random_forest_v2.tune(model_params=model_params)

Displaying top 10 fitted estimators

In [ ]:
grid_search_results_summary_best_four, grid_search_pipelines_best_four = random_forest_v2.searches[random_forest_v2.indices[-1]].score_summary()

In [ ]:
grid_search_results_summary_best_four.head(10)

Compared to the last search, the scores are lower for this smaller group of more important features. This however is desired to avoid over-fitting and bias towards the train set.

**Retrieving the best hyperparameter combination.**

In [ ]:
best_model = grid_search_results_summary_best_four.iloc[0, 0]
best_model_params = grid_search_pipelines_best_four[best_model].best_params_
print('Best model:', best_model)
print('Best hyperparameter combination:', best_model_params)

In [ ]:
random_forest_v2.best_regressors[random_forest_v2.indices[-1]]

**Extracting the feature importance**

In [ ]:
feature_importances_best_four_df = random_forest_v2.extract_feature_importance()

The 'OverallQual' feature still dominates, with the remaining features having a combined importance value of about ~40%. Does seem to suggest that 'OverallQual' is the most significant feature for the sale price prediction; which was predicted during the sale price correlation study. 

Evaluating model on the train and test sets

In [ ]:
random_forest_v2.evaluate_model_performance()

It can be seen from the plots and the $R^2$ values that the model has significantly improved in predicting the test set target values, going from $R^2\approx0.49$ to $R^2\approx0.74$.
At the same the train set performance has declined a fair bit to $R^2\approx0.75$, but as mentioned before this is a consequence of reducing the degree of over-fitting. 

Despite the improvement in performance on the test set, the model performance is marginally below the success criterion of $R^2=0.75$

**Possible next steps**:
* Try an even smaller group of features, or a different combination of the most important features.
* Perform hyperparameter tuning with a new range of hyperparamter values.
* Make adjustments to the data cleaning and feature engineering pipeline.

---

## Refitting with 3 most important features

In [ ]:
best_three_features = feature_importances_df.iloc[0:3,:].index.values.tolist()
features_to_drop = x_train.drop(best_three_features, axis=1).columns.tolist()
print('Best three features:', best_three_features)
print('features to drop:', features_to_drop)

In [ ]:
from feature_engine.selection import DropFeatures
import src.ml.transformers_and_functions as tf


def data_cleaning_and_feature_engineering_v3():
    """
    Constructs and returns data cleaning and feature engineering pipeline.
    """
    # variables for defining pipeline
    estimator = DecisionTreeRegressor(min_samples_split=10, min_samples_leaf=5, random_state=30)
    # Orginally intended to include the categories parameter used previously for OrdinalEncoder in the
    # feature engineering notebook. However causes problems when not all category options are
    # present in the train or test set. Will use the 'auto' option instead. 

    pipeline = Pipeline([
                        # Data cleaning:
                        # Missing value imputation:
                        ('IndependentKNNImputer', tf.IndependentKNNImputer()),
                        ('EqualFrequencyImputer', tf.EqualFrequencyImputer()),
                        #feature engineering:
                        # encoding:
                        ('OrdinalEncoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype='int64')),
                        # feature number reduction
                        ('DropFeatures', DropFeatures(features_to_drop=features_to_drop)),
                        # feature scaling:
                        ('CompositeNormaliser', tf.CompositeNormaliser())
                        ])
    return pipeline

Creating MLModel instance with the new data cleaning and engineering pipeline

In [ ]:
random_forest_v3 = MLModel(cleaning_engineering_pipeline=data_cleaning_and_feature_engineering_v3(), estimator_name='RandomForestRegressor',
                           estimator_instance=RandomForestRegressor(random_state=30), x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

Redo tuning with only the 3 best features

In [ ]:
random_forest_v3.tune(model_params=model_params)

Displaying the top 10 fitted estimators

In [ ]:
grid_search_results_summary_best_three, grid_search_pipelines_best_three = random_forest_v3.searches[random_forest_v3.indices[-1]].score_summary()

In [ ]:
grid_search_results_summary_best_three.head(10)

The validation scores of the best regressors are worse than for the previous four most important feature model

In [ ]:
best_model = grid_search_results_summary_best_three.iloc[0, 0]
best_model_params = grid_search_pipelines_best_three[best_model].best_params_
print('Best model:', best_model)
print('Best hyperparameter combination:', best_model_params)

In [ ]:
random_forest_v3.best_regressors[random_forest_v3.indices[-1]]

In [ ]:
feature_importances_best_three_df = random_forest_v3.extract_feature_importance()

Evaluating model on the train and test sets


In [ ]:
random_forest_v3.evaluate_model_performance()

The $R^2$ value is slightly worse for both the test and train set relative to the previous four most important feature model. Thus it appears reducing the number of features in the model further does not improve the model performance.

---

## Studying the validation curves

Using the current best 4 feature model will look at the validation curves for each hyperparameter to see how altering the parameters further may or may not significantly
improve the model performance.

Of course a limitation inherent to varying one parameter whilst keep others fixed, is that the degree of variation in model performance is dependent on the values of
the fixed parameters. The model performance is a multivariable function of the hyperparameters, that can be visualised as a multidimensional surface whose height is the
model performance score. Constraining all but one parameter effectively reduces movement along the surface to a single contour for which the hyperparameter values are constant. The exact contour will vary with the values of the fixed parameters. As such a global maximum in the surface height may not be discovered unless the contour passes through the surface at this point. Nonetheless it is hoped that the previous grid search tuning means that the hyperparameter values are close to their optimal values, such that the contour does pass through any global maximum.

In [ ]:
best_four_features = feature_importances_df.iloc[0:4,:].index.values.tolist()
features_to_drop = x_train.drop(best_four_features, axis=1).columns.tolist()
print('Best four features:', best_four_features)
print('features to drop:', features_to_drop)

Reminder of the current best regressor parameters for the 4 feature model

In [ ]:
random_forest_v2.best_regressors['0']

### Generating a validation curve for the max_depth parameter

In [ ]:
x_train_copy = x_train.copy(deep=True)
y_train_copy = y_train.copy(deep=True)
y_train_copy = scale_target(y_train_copy, y_train_copy)[0]
x_train_copy = random_forest_v2.cleaning_engineering_pipeline.fit(x_train_copy, y_train_copy).transform(x_train_copy)

In [ ]:
from sklearn.model_selection import validation_curve
train_scores, validation_scores = validation_curve(estimator=random_forest_v2.best_regressors['0'], X=x_train_copy, y=y_train_copy, param_name='max_depth',
                                             param_range=np.arange(1,25,1), cv=5, scoring='r2', n_jobs=-1)

In [ ]:
fig, ax = plt.subplots()
x = np.arange(1,25,1)
y1 = np.mean(train_scores, axis=1)
y2 = np.mean(validation_scores, axis=1)
ax.set_xticks(x)
ax.set(title='Validation curve for max_depth parameter')
ax.plot(x, y1, marker='o', label='training score')
ax.plot(x, y2, marker='o', label='validation score')
ax.legend(loc='center')
ax.fill_between(x, y1 + np.std(train_scores, axis=1), y1 - np.std(train_scores, axis=1), alpha=0.4)
ax.fill_between(x, y2 + np.std(validation_scores, axis=1), y2 - np.std(validation_scores, axis=1), alpha=0.4)

The training and validation scores are fairly similar, but there is some overfitting. The model performance plateaus after a max depth value of 6.

### Generating a validation curve for the min_samples_leaf parameter

In [ ]:
train_scores, validation_scores = validation_curve(estimator=random_forest_v2.best_regressors['0'], X=x_train_copy, y=y_train_copy, param_name='min_samples_leaf',
                                             param_range=np.arange(1,30,1), cv=5, scoring='r2', n_jobs=-1)

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
x = np.arange(1,30,1)
y1 = np.mean(train_scores, axis=1)
y2 = np.mean(validation_scores, axis=1)
ax.set_xticks(x)
ax.set(title='Validation curve for min_samples_leaf parameter')
ax.plot(x, y1, marker='o', label='training score')
ax.plot(x, y2, marker='o', label='validation score')
ax.legend(loc='upper right')
ax.fill_between(x, y1 + np.std(train_scores, axis=1), y1 - np.std(train_scores, axis=1), alpha=0.4)
ax.fill_between(x, y2 + np.std(validation_scores, axis=1), y2 - np.std(validation_scores, axis=1), alpha=0.4)

The training and validation scores are fairly similar for all but small values where there is some overfitting. The model performance seems to decrease after the
min_samples_leaf values of 2 or less.

### Generating a validation curve for the min_samples_split parameter

In [ ]:
train_scores, validation_scores = validation_curve(estimator=random_forest_v2.best_regressors['0'], X=x_train_copy, y=y_train_copy, param_name='min_samples_split',
                                             param_range=np.arange(2,101,2), cv=5, scoring='r2', n_jobs=-1)

In [ ]:
fig, ax = plt.subplots(figsize=(12,7), tight_layout=True)
x = np.arange(2,101,2)
y1 = np.mean(train_scores, axis=1)
y2 = np.mean(validation_scores, axis=1)
ax.set_xticks(x)
ax.set(title='Validation curve for min_samples_split parameter')
ax.plot(x, y1, marker='o', label='training score')
ax.plot(x, y2, marker='o', label='validation score')
ax.legend(loc='upper center')
ax.fill_between(x, y1 + np.std(train_scores, axis=1), y1 - np.std(train_scores, axis=1), alpha=0.4)
ax.fill_between(x, y2 + np.std(validation_scores, axis=1), y2 - np.std(validation_scores, axis=1), alpha=0.4)

The training and validation scores are again fairly similar, but there is some overfitting for smaller values. The model performance decreases after a min_samples_split value of around 16.

### Generating a validation curve for the max_leaf_nodes parameter

In [ ]:
train_scores, validation_scores = validation_curve(estimator=random_forest_v2.best_regressors['0'], X=x_train_copy, y=y_train_copy, param_name='max_leaf_nodes',
                                             param_range=np.arange(0, 151, 5), cv=5, scoring='r2', n_jobs=-1)

In [ ]:
fig, ax = plt.subplots(figsize=(13,7))
x = np.arange(0, 151, 5)
y1 = np.mean(train_scores, axis=1)
y2 = np.mean(validation_scores, axis=1)
ax.set_xticks(x)
ax.set(title='Validation curve for max_leaf_nodes parameter')
ax.plot(x, y1, marker='o', label='training score')
ax.plot(x, y2, marker='o', label='validation score')
ax.legend(loc='center')
ax.fill_between(x, y1 + np.std(train_scores, axis=1), y1 - np.std(train_scores, axis=1), alpha=0.4)
ax.fill_between(x, y2 + np.std(validation_scores, axis=1), y2 - np.std(validation_scores, axis=1), alpha=0.4)

The training and validation scores are fairly similar only for very small values, after there is noticeable overfitting. The model performance plateaus after a max_leaf_nodes
value of around 25.

### Generating a validation curve for the max_samples parameter

In [ ]:
train_scores, validation_scores = validation_curve(estimator=random_forest_v2.best_regressors['0'], X=x_train_copy, y=y_train_copy, param_name='max_samples',
                                             param_range=np.arange(0.1, 1.05, 0.05), cv=5, scoring='r2', n_jobs=-1)

In [ ]:
fig, ax = plt.subplots(figsize=(9,7))
x = np.arange(0.1, 1.05, 0.05)
y1 = np.mean(train_scores, axis=1)
y2 = np.mean(validation_scores, axis=1)
ax.set_xticks(x)
ax.set(title='Validation curve for max_samples parameter')
ax.plot(x, y1, marker='o', label='training score')
ax.plot(x, y2, marker='o', label='validation score')
ax.legend(loc='upper left')
ax.fill_between(x, y1 + np.std(train_scores, axis=1), y1 - np.std(train_scores, axis=1), alpha=0.4)
ax.fill_between(x, y2 + np.std(validation_scores, axis=1), y2 - np.std(validation_scores, axis=1), alpha=0.4)

The training and validation scores are fairly similar again only for small values, and after there is some overfitting. The model performance is slowly plateaus after a value of 0.3 for the validation score, whilst unsurprisingly it continues to increases for the training score.

### Generating a validation curve for the n_estimators parameter

In [ ]:
train_scores, validation_scores = validation_curve(estimator=random_forest_v2.best_regressors['0'], X=x_train_copy, y=y_train_copy, param_name='n_estimators',
                                             param_range=np.arange(10, 351, 10), cv=5, scoring='r2', n_jobs=-1)

In [ ]:
fig, ax = plt.subplots(figsize=(13,7), tight_layout=True)
x = np.arange(10, 351, 10)
y1 = np.mean(train_scores, axis=1)
y2 = np.mean(validation_scores, axis=1)
ax.set_xticks(x)
ax.set(title='Validation curve for n_estimators parameter')
ax.plot(x, y1, marker='o', label='training score')
ax.plot(x, y2, marker='o', label='validation score')
ax.legend(loc='lower right')
ax.fill_between(x, y1 + np.std(train_scores, axis=1), y1 - np.std(train_scores, axis=1), alpha=0.4)
ax.fill_between(x, y2 + np.std(validation_scores, axis=1), y2 - np.std(validation_scores, axis=1), alpha=0.4)

There is significant overfitting. The model performance plateaus after a value of 150.

### Generating a validation curve for the max_features parameter

In [ ]:
train_scores, validation_scores = validation_curve(estimator=random_forest_v2.best_regressors['0'], X=x_train_copy, y=y_train_copy, param_name='max_features',
                                             param_range=[1,2,3,4], cv=5, scoring='r2', n_jobs=-1)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7), tight_layout=True)
x = [1,2,3,4]
y1 = np.mean(train_scores, axis=1)
y2 = np.mean(validation_scores, axis=1)
ax.set_xticks(x)
ax.set(title='Validation curve for max_features parameter')
ax.plot(x, y1, marker='o', label='training score')
ax.plot(x, y2, marker='o', label='validation score')
ax.legend(loc='upper left')
ax.fill_between(x, y1 + np.std(train_scores, axis=1), y1 - np.std(train_scores, axis=1), alpha=0.4)
ax.fill_between(x, y2 + np.std(validation_scores, axis=1), y2 - np.std(validation_scores, axis=1), alpha=0.4)

There is significant overfitting. The model performance peaks for a value of 2 for the validation set.

### General comments
- For most validation curves, the validation scores have a large degree of variance for all parameter values.
- All curves plateau/peak quickly, and only a few decline rapidly.
- Also the current best model parameter values achieve close to the highest scoring values for most parameters, however for the parameters max_samples, min_samples_split, max_leaf_nodes, and min_samples_leaf, some improvements to the model performance may be possible with slight tweaks to these parameter values.



Changing one of these parameters will change the location on the hyperparameter surface, and so the improvements implied by tweaking the remaining parameters may no longer be the same. Thus rather than just changing all parameters simultaneously, a small Grid Search tuning will be performed with a mixture of the original and tweaked parameters.

### Validation curve inspired further grid search tuning

Creating MLModel instance

In [ ]:
random_forest_v4 = MLModel(cleaning_engineering_pipeline=data_cleaning_and_feature_engineering_v2(), estimator_name='RandomForestRegressor',
                           estimator_instance=RandomForestRegressor(random_state=30), x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

In [ ]:
model_params = {'RandomForestRegressor': {
    'max_depth': [10],
    'max_samples': [0.5,1.0],
    'max_features': ['sqrt'],
    'n_estimators': [150],
    'min_samples_leaf': [2,5],
    'min_samples_split': [2,4,12],
    'max_leaf_nodes': [25,32]
}}

In [ ]:
random_forest_v4.tune(model_params=model_params)

In [ ]:
grid_search_results_summary, grid_search_pipelines  = random_forest_v4.searches[random_forest_v4.indices[-1]].score_summary()

In [ ]:
grid_search_results_summary.head(10)

It seems tweaking the max_samples and min_samples_leaf parameters is responsible for an improved best regressor validation set score.

In [ ]:
best_model = grid_search_results_summary.iloc[0, 0]
best_model_params = grid_search_pipelines[best_model].best_params_
print('Best model:', best_model)
print('Best hyperparameter combination:', best_model_params)

In [ ]:
random_forest_v4.best_regressors['0']

Evaluating model on the train and test sets

In [ ]:
random_forest_v4.evaluate_model_performance()

It can be seen from the plots and the $R^2$ values that the model has slightly improved in predicting the test set target values, going from $R^2\approx0.741$ to $R^2\approx0.751$.
At the same the train set performance has improved a bit to $R^2\approx0.79$. 

With this improvement in performance, the model performance on both the train and test set meets the success criterion of $R^2>0.75$

---

## Current best regressor

The best regressor is currently the model fitted with the 4 most important features, with a test set score of $R^2=0.751$. This meets the success criteria.

In [ ]:
random_forest_v4.best_regressors['0']

In [ ]:
best_model = grid_search_results_summary.iloc[0, 0]
best_model_params = grid_search_pipelines[best_model].best_params_
print('Best model:', best_model)
print('Best hyperparameter combination:', best_model_params)

With feature importance:

In [ ]:
random_forest_v4.extract_feature_importance(0)

And model performance:

In [ ]:
random_forest_v4.evaluate_model_performance(0)

---

## Saving the current best pipelines, feature importances df, and the model performances df

In [ ]:
model_pipeline = random_forest_v4.model_pipelines['0']
data_cleaning_and_feature_engineering_pipeline = random_forest_v4.cleaning_engineering_pipeline
feature_importances_df = random_forest_v4.extract_feature_importance(print_plot=False)
model_performances_df = random_forest_v4.model_performances_df

In [ ]:
try:
    joblib.dump(value=model_pipeline, filename=f"{file_path}/model_pipeline.pkl")
except Exception as e:
    print(e)

In [ ]:
try:
    joblib.dump(value=data_cleaning_and_feature_engineering_pipeline, filename=f"{file_path}/data_cleaning_and_feature_engineering_pipeline.pkl")
except Exception as e:
    print(e)

In [ ]:
try:
    feature_importances_df.to_csv(f"{file_path}/feature_importances_df.csv")
except Exception as e:
    print(e)

In [ ]:
try:
    model_performances_df.to_csv(f"{file_path}/model_performances_df.csv")
except Exception as e:
    print(e)

### Saving the train and test sets

In [ ]:
try:
    train_set_df.to_csv(f"{file_path}/train_set_df.csv")
except Exception as e:
    print(e)

In [ ]:
try:
    test_set_df.to_csv(f"{file_path}/test_set_df.csv")
except Exception as e:
    print(e)